In [ ]:

from datetime import datetime
from glob import glob
import json
import os
import re
from pypdf import PdfReader 
import pandas as pd
import calendar

# last_salt_type = None
# last_salinity = None
# last_weight_material_density = None

''' Notes
unit of weight material density for che is kg/m3
unit of low gravity solid percent is percent, and only use the first float in the front of '/'
unit of salinity is a percent and only use the first float in the front of '/'
'''

def convert_lbbbl_2_kgm3(lbbbl: float):
    return lbbbl * 2.85

# for alaska wells
def get_base_fluid_type(mud_class):
    if mud_class == 'waterbased':
        return 'Water-Based Mud'
    elif mud_class == 'oilbased':
        return 'Escaid 110'
    else:
        raise Exception()

def merge_fluid_properties(
        report_path,
        oil_water_ratio,
        salt_type,
        salinity,
        low_gravity_density,
        weight_material_density
          ):
    with open(report_path, 'r') as f:
        report = json.load(f)

    assert len(report['fluid']) == 1

    fluid_witsml_obj = report['fluid'][0]

    assert 'tempVis' in fluid_witsml_obj.keys()
    assert 'WaterPhaseSalinity' not in fluid_witsml_obj.keys()
    assert 'SolidsLowGravPc' not in fluid_witsml_obj.keys()

    for i, extension_name_value in enumerate(fluid_witsml_obj['extensionNameValue']):
        assert extension_name_value['name'] != 'MudInTemperature'
        assert extension_name_value['name'] != 'BaseFluid'
        if extension_name_value['name'] == 'SaltType':
            fluid_witsml_obj['extensionNameValue'].remove(extension_name_value)
        assert extension_name_value['name'] != 'WeightMaterialDensity'


    oil_water_ratio_strs = oil_water_ratio.split('/')
    oil_pc = float(oil_water_ratio_strs[0]) / 100
    water_pc = float(oil_water_ratio_strs[1]) / 100

    fluid_witsml_obj['oilPc'] = {
                'value': oil_pc
            }
    fluid_witsml_obj['waterPc'] = {
                'value': water_pc 
            }
    fluid_witsml_obj['waterPhaseSalinity'] = {
        'value': salinity
    }
    fluid_witsml_obj['solidsLowGravPc'] = {
        'value': low_gravity_density
    }

    # if '/' in str(weight_material_density):
    #     weight_material = weight_material_density.split('/')
    #     weight_material = float(weight_material[1]) if len(weight_material)>1 else 0
    # else:
    #     weight_material = float(weight_material_density)

    fluid_witsml_obj['extensionNameValue'] +=[
        {
            'name': 'MudInTemperature',
            'value': fluid_witsml_obj['tempVis']
        },
        {
            'name': 'BaseFluid',
            'value': {
                'value': get_base_fluid_type(fluid_witsml_obj['mudClass'])
            }
        },
        {
            "name": "SaltType",
            "value": {
                "value": salt_type
            },
        },
        {
            'name': 'WeightMaterialDensity',
            'value': {
                'value': weight_material_density
            }
        }
    ]

    return report

def get_DMR_info_given_date(date: datetime, DMR_base_dir: str):
    # global last_salinity
    # global last_salt_type
    # global last_weight_material_density

    pdf_paths = glob(os.path.join(DMR_base_dir, '*.pdf'))
    date_sub_str = f'{date.month}-{date.day}-{str(date.year)[-2:]}'
    matched_pdf = list(filter(lambda x: date_sub_str in x, pdf_paths))

    if len(matched_pdf) != 1:
        print(f'Date: {date}')
        print(f'Reason: find DMR pdf')
        print(f'Will use the first pdf, Please check the output json!!!')

    DMR_raw_text = PdfReader(matched_pdf[0]).pages[0].extract_text()
    # not neccessary
    # solid_percent = re.compile(rf'Solids %vol ([0-9]+\.*[0-9]*)').findall(DMR_raw_text)

    # Oil/Water %vol 1.25/94.25
    oil_water_ratio = re.compile(rf'Oil/Water %vol (\d+\.?\d*/\d+\.?\d*)').findall(DMR_raw_text)
    if len(oil_water_ratio) == 0:
        oil_water_ratio = re.compile(r"Oil/Water Ratio (\d+/\d+)").findall(DMR_raw_text)[:-1]

    if len(oil_water_ratio) != 1:
        oil_water_ratio = None
        print(f'Date: {date}')
        print(f'Reason: oil water ratio')
        print(f'Pdf path: {matched_pdf[0]}')
        print(f'Will set oil water ratio as None for default !!!')

    # assert len(oil_water_ratio) <= 1
    # if len(oil_water_ratio) != 1:
    #     oil_water_ratio = re.compile(rf'Oil/Water Ratio (\d+\.?\d*/\d+\.?\d*)').findall(DMR_raw_text)
    # assert len(oil_water_ratio) >= 1

    nacl_ratio = re.compile(rf'NaCl %vol / lb/bbl (\d+\.?\d* / \d+\.?\d*)').findall(DMR_raw_text)
    kcl_ratio = re.compile(rf'KCl %vol / lb/bbl (\d+\.?\d* / \d+\.?\d*)').findall(DMR_raw_text)

    if len(nacl_ratio) == 1 or len(kcl_ratio) == 1:
        if len(nacl_ratio) == 0:
            salt_type = 'KCl'
            salinity = kcl_ratio
        elif len(kcl_ratio) == 0:
            salt_type = 'NaCl'
            salinity = nacl_ratio
        else:
            if eval(nacl_ratio[0]) > eval(kcl_ratio[0]):
                salt_type = 'NaCl'
                salinity = nacl_ratio
            else:
                salt_type = 'KCl'
                salinity = kcl_ratio
        salinity = float(salinity[0].split('/')[0].strip()) / 100
    else:
        salt_type = 'CaCl2'
        salinity = re.compile(r'Salt Wt %wt (\d+)').findall(DMR_raw_text)
        assert len(salinity) == 1
        salinity = float(salinity[0])
        salinity /= 100
        # print(f'Date: {date}')
        # print(f'Reason: salt type and salinity')
        # print(f'Pdf path: {matched_pdf[0]}')
        # raise Exception()

    low_gravity_density = re.compile(rf'Low Gravity % / lb/bbl (\d+\.?\d* / \d+\.?\d*)').findall(DMR_raw_text)
    # low_gravity_density_2 = re.compile(rf'Low Gravity  % (\d+\.\d+)').findall(DMR_raw_text)
    if len(low_gravity_density) == 1:
        lgs_density = float(low_gravity_density[0].split('/')[0]) / 100
    else:
        lgs_density = re.compile(r'Low Gravity  % (\d+.\d+)').findall(DMR_raw_text)
        if len(lgs_density) == 0:

            print(f'Date: {date}')
            print(f'Reason: lgs density')
            print(f'Pdf path: {matched_pdf[0]}')
            print(f'Will set lgs density as None for default !!!')
            lgs_density = None
            
        lgs_density = float(lgs_density[0])

    weight_material_density = re.compile(rf'Weight Material % / lb/bbl (\d+\.?\d* / \d+\.?\d*)').findall(DMR_raw_text)
    if len(weight_material_density) == 0:
        # print(f'Date: {date}')
        # print(f'Reason: weight material density')
        # print(f'Pdf path: {matched_pdf[0]}')
        # raise Exception()
        print(f'Will set weight material density as 4100 for default !!!')
        weight_material_density = 4100 # kg/m3
    else:
        weight_material_density = float(weight_material_density[0].split('/')[1]) * 2.85

    return oil_water_ratio[0], salt_type, salinity, lgs_density, weight_material_density
        
def find_sheet(partial_name):
    ss = []
    for s in dmr_sheet_names:
        if partial_name in s:
            ss.append(s)

    assert len(ss) == 1

    return ss[0]

def get_nacl_ratio(row):
    for k, v in row.items():
        nacl_ratio = re.compile(rf'NaCl').findall(str(v))
        if len(nacl_ratio) > 0:
            print(nacl_ratio)
    # nacl_ratio = re.compile(rf'NaCl %vol / lb/bbl (\d+\.?\d* / \d+\.?\d*)')
    # kcl_ratio = re.compile(rf'KCl %vol / lb/bbl (\d+\.?\d* / \d+\.?\d*)').findall(DMR_raw_text)


well_names= ['Dancer_run1_bha', 'Dancer_run2_bha','Dancer_run3_bha','Dancer_run4_bha','Dancer_run5_bha',]
DMR_excel_path = r"C:\NotOneDrive\Data\algo_data_nas_copy\Dancer\Santos-Dancer 1_For Replay\Well Dancer-1 Carnarvon Basin  Santos.xlsx"

algo_input_base_dir = r"C:\NotOneDrive\Data\algo_seperate_data"


dmr = pd.read_excel(DMR_excel_path, sheet_name=None)
dmr = pd.ExcelFile(DMR_excel_path)
dmr_sheet_names = dmr.sheet_names

for well_name in well_names:
    mapping_file_path = os.path.join(algo_input_base_dir, well_name, 'mapping.csv')

    mapping_content = pd.read_csv(mapping_file_path, names=['time_key', 'file_name'])
    mapping_content['time_key'] = pd.to_datetime(mapping_content['time_key'])

    for index , row in mapping_content.iterrows():
        if 'FluidsReport' not in row['file_name']:
            continue
        date_time = pd.to_datetime(row['time_key'])
        file_name = row['file_name']
        date_for_dmr = f'{date_time.day}'.zfill(2) + rf'-{calendar.month_abbr[date_time.month]}-{date_time.year}'
        print(date_for_dmr)
        sheet_name = find_sheet(date_for_dmr)
        sheet = pd.read_excel(DMR_excel_path, sheet_name=sheet_name)
        for i, row in sheet.iterrows():
            get_nacl_ratio(row)
        # sheet.to_excel(f'temp_{date_for_dmr}.xlsx')


    #     report_path = os.path.join(algo_input_base_dir, well_name, row['file_name'])

    #     merged_report = merge_fluid_properties(
    #         report_path,
    #         oil_water_ratio,
    #         salt_type,
    #         salinity,
    #         low_gravity_density,
    #         weight_material_density
    #     )

        # merged_report_save_path = os.path.join(algo_input_base_dir, well_name, fr'more_{row["file_name"]}')
        # with open(merged_report_save_path, 'w') as f:
        #     json.dump(merged_report, f, indent=4)
